In [3]:
search_for = 126
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002153635025024414
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 375
all cases: 956096141
type: [1, 1, 1, 1, 126] 126
cases of this type: 252047376
100000 23.69634740374041
200000 40.16144160043599
300000 44.78585820652657
400000 50.86285991563881
500000 54.19504316817631
600000 52.972412467160325
700000 50.82141160152406
800000 52.33698343247139
900000 53.08135699953992
1000000 43.891377050114144
1100000 36.64932422153687
1200000 56.67766755914086
1300000 52.22357742512627
1400000 41.69778871093118
1500000 50.1709807763025
1600000 58.25833629827692
1700000 50.752061147140566
1800000 57.39915223050309
1900000 49.62211237494839
2000000 31.908065272544228
2100000 24.698018065720863
2200000 56.07248206188239
2300000 43.440852276356665
2400000 56.84164440862965
2500000 56.48683701329809
2600000 64.2822058711108
2700000 36.07663258906737
2800000 45.15427568305999
2900000 39.20723526258461
3000000 46.33720977007

30100000 75.96694981568639
30200000 74.29303496098387
30300000 30.763380558409054
30400000 72.60355510720791
30500000 73.20775747990251
30600000 71.78780028935569
30700000 72.86302868609117
30800000 66.83187410079965
30900000 67.30495328791031
31000000 53.32196460552527
31100000 41.188010286223154
31200000 64.18299963208572
31300000 68.08553155895532
31400000 70.29326615505802
31500000 69.64006586308317
31600000 72.52292767896412
31700000 74.87606131664815
31800000 40.862566324436365
31900000 66.95702074084349
32000000 75.08426925142483
32100000 77.9858698780772
32200000 74.27467885513612
32300000 37.44935796613039
32400000 71.45309434671609
32500000 75.09713186829886
32600000 70.79177872764673
32700000 74.55887989140308
32800000 68.63703508680592
32900000 70.40766127585191
33000000 53.94909519399138
33100000 41.26826542518863
33200000 67.56748092909075
33300000 70.20425961078053
33400000 71.67439612967924
33500000 72.68069673699459
33600000 74.54500868794221
33700000 78.74507481180889

60500000 47.09527877305484
60600000 50.99748895131805
60700000 62.54592029831406
60800000 63.592325925227
60900000 65.95326206734474
61000000 57.87176577296954
61100000 33.623291154515194
61200000 66.75189996044705
61300000 64.90927333546175
61400000 66.75960691575274
61500000 65.52280868143905
61600000 33.877162811884034
61700000 72.93987954650407
61800000 71.22873735065552
61900000 72.36171444333458
62000000 76.17860579883595
62100000 76.24484692067125
62200000 69.17380689560989
62300000 69.52834501929867
62400000 70.0208069959269
62500000 59.99376562670847
62600000 40.1400918819484
62700000 60.86805514116102
62800000 63.43017088736453
62900000 61.14930295389996
63000000 52.433156530049175
63100000 34.3032978060046
63200000 61.849606628639
63300000 64.64927309758865
63400000 63.85180203323749
63500000 58.437048737313894
63600000 41.069579168486754
63700000 64.46574183023735
63800000 67.28019759121699
63900000 66.48052148829149
64000000 76.30246374885768
64100000 72.82570400754321
642

90900000 58.604573833292235
91000000 52.53704715775115
91100000 31.433900386783623
91200000 56.62503455259861
91300000 48.68969942683672
91400000 41.89855537805538
91500000 61.69509322798542
91600000 58.74592024188919
91700000 59.514230777777385
91800000 70.61620173895793
91900000 64.05405800755116
92000000 66.34426475405888
92100000 65.93898427982008
92200000 68.5889089377884
92300000 63.90721647963277
92400000 64.44143545533423
92500000 61.421492274425475
92600000 62.259517098880714
92700000 61.54461345368813
92800000 28.117206974549163
92900000 60.11065255241001
93000000 52.47840321284058
93100000 30.54532694081541
93200000 58.5767988980187
93300000 41.904475882097245
93400000 47.438382871801
93500000 59.73861046570709
93600000 57.740625979482836
93700000 61.34527217895614
93800000 68.2098875259186
93900000 64.76782240816807
94000000 68.19183803683474
94100000 66.77740236532561
94200000 65.73589408038562
94300000 62.63079566070049
94400000 64.94710400808171
94500000 64.4005468433879

120300000 75.59586853249247
120400000 70.96725350659675
120500000 72.38943855148442
120600000 72.07961482456919
120700000 70.6420251440334
120800000 70.79733238870016
120900000 70.88122748035208
121000000 38.86786819465386
121100000 10.788681148033994
121200000 62.99725641248822
121300000 70.55915659611779
121400000 71.56630207905314
121500000 72.53932621679024
121600000 73.47630299095424
121700000 74.69158111601425
121800000 75.7370742804359
121900000 75.0708995632535
122000000 82.09405074065117
122100000 24.3858667564749
122200000 6.6610789119182865
122300000 6.263741561117451
122400000 6.370748335499517
122500000 6.419975674367215
122600000 7.101225258235876
122700000 6.648312483947655
122800000 6.717154698715222
122900000 6.477380501246434
123000000 7.396815621464767
123100000 7.114758417687673
123200000 6.3860621871985135
123300000 6.492481012815482
123400000 6.9672309477688605
123500000 6.574146167217785
123600000 6.5573759694375955
123700000 6.435932015533823
123800000 6.2962472

...

884600000 3.334176646998686
884700000 2.413270843240559
884800000 3.39525414459217
884900000 2.136570791246629
885000000 3.167901843732004
885100000 2.2443735159953615
885200000 2.975557896673822
885300000 3.0811307345029935
885400000 2.292995150540723
885500000 3.163251570973516
885600000 2.048439709782213
885700000 3.409180285769716
885800000 1.6924407324776292
885900000 3.214921596932728
886000000 1.7444448397628975
886100000 2.908546444643378
886200000 2.5550987164725063
886300000 2.5403046088602093
886400000 3.9161452002293564
886500000 2.4159997152503445
886600000 1.654716733092117
886700000 0.5397790064302226
886800000 0.5265614214232306
886900000 0.5251264757252666
887000000 0.5279493053772389
887100000 0.5266413263721121
887200000 0.5229111130794181
887300000 0.5245484101797462
887400000 0.5236581905946931
887500000 0.5269929618411295
887600000 0.5263182008680741
887700000 2.6980813441478495
887800000 2.3919341976724895
887900000 4.156708968999196
888000000 2.0492199447517727


909200000 0.3541835973098338
best so far: 0
type: [1, 3, 6, 7, 1] 126
cases of this type: 37044
best so far: 0
type: [1, 3, 7, 1, 6] 126
cases of this type: 190512
909300000 1.0076468529116942
909400000 0.4864939650667852
best so far: 0
type: [1, 3, 7, 2, 3] 126
cases of this type: 95256
909500000 0.36121140079130526
best so far: 0
type: [1, 3, 7, 3, 2] 126
cases of this type: 63504
909600000 0.33864384238230255
best so far: 0
type: [1, 3, 7, 6, 1] 126
cases of this type: 31752
best so far: 0
type: [1, 3, 14, 1, 3] 126
cases of this type: 47628
909700000 0.33027022026501235
best so far: 0
type: [1, 3, 14, 3, 1] 126
cases of this type: 15876
best so far: 0
type: [1, 3, 21, 1, 2] 126
cases of this type: 21168
best so far: 0
type: [1, 3, 21, 2, 1] 126
cases of this type: 10584
best so far: 0
type: [1, 3, 42, 1, 1] 126
cases of this type: 5292
best so far: 0
type: [1, 6, 1, 1, 21] 126
cases of this type: 1166886
909800000 0.6396556470629772
909900000 0.9222713658587204
910000000 0.90045809

920800000 1.4516306177549478
920900000 1.0404684054316342
921000000 1.2771616773598617
921100000 1.4538460394277892
921200000 0.9251012125032841
921300000 1.4140879490310114
921400000 0.9790679473508239
921500000 1.280887568683207
921600000 1.4078338994479491
921700000 0.9381368187260919
921800000 1.458390656878682
921900000 1.1216575977064962
922000000 1.454043529658739
922100000 0.4246319404118478
922200000 0.2556290336125963
922300000 0.2556148386482007
922400000 0.25593745515992244
922500000 0.25549893062386114
922600000 0.24962901025110745
922700000 0.24524494378478726
922800000 0.2460261029092325
922900000 0.2448559876306878
923000000 0.24349534458298086
923100000 0.24596459555579753
923200000 0.24891840158083706
923300000 0.7776168499437689
923400000 1.1782634066293007
923500000 1.4659145334864108
923600000 1.3884312873944402
923700000 0.8886069637693823
923800000 1.319036573168385
923900000 0.9252420385838925
924000000 1.1477390419648092
924100000 1.292759786962004
924200000 0.

944700000 0.2246293698795226
best so far: 0
type: [2, 3, 1, 3, 7] 126
cases of this type: 194481
944800000 0.1918131013300419
944900000 0.13846280384344659
best so far: 0
type: [2, 3, 1, 7, 3] 126
cases of this type: 83349
945000000 0.12988940459503928
best so far: 0
type: [2, 3, 1, 21, 1] 126
cases of this type: 27783
best so far: 0
type: [2, 3, 3, 1, 7] 126
cases of this type: 64827
945100000 0.10757088250602352
best so far: 0
type: [2, 3, 3, 7, 1] 126
cases of this type: 9261
best so far: 0
type: [2, 3, 7, 1, 3] 126
cases of this type: 11907
best so far: 0
type: [2, 3, 7, 3, 1] 126
cases of this type: 3969
best so far: 0
type: [2, 3, 21, 1, 1] 126
cases of this type: 1323
best so far: 0
type: [2, 7, 1, 1, 9] 126
cases of this type: 45927
best so far: 0
type: [2, 7, 1, 3, 3] 126
cases of this type: 15309
945200000 0.10703963150880137
best so far: 0
type: [2, 7, 1, 9, 1] 126
cases of this type: 5103
best so far: 0
type: [2, 7, 3, 1, 3] 126
cases of this type: 5103
best so far: 0
type:

955400000 0.009306120256859063
955500000 0.006848523804996411
best so far: 0
type: [6, 1, 1, 3, 7] 126
cases of this type: 64827
best so far: 0
type: [6, 1, 1, 7, 3] 126
cases of this type: 27783
955600000 0.005060899327423175
best so far: 0
type: [6, 1, 1, 21, 1] 126
cases of this type: 9261
best so far: 0
type: [6, 1, 3, 1, 7] 126
cases of this type: 21609
best so far: 0
type: [6, 1, 3, 7, 1] 126
cases of this type: 3087
best so far: 0
type: [6, 1, 7, 1, 3] 126
cases of this type: 3969
best so far: 0
type: [6, 1, 7, 3, 1] 126
cases of this type: 1323
best so far: 0
type: [6, 1, 21, 1, 1] 126
cases of this type: 441
best so far: 0
type: [6, 3, 1, 1, 7] 126
cases of this type: 7203
best so far: 0
type: [6, 3, 1, 7, 1] 126
cases of this type: 1029
best so far: 0
type: [6, 3, 7, 1, 1] 126
cases of this type: 147
best so far: 0
type: [6, 7, 1, 1, 3] 126
cases of this type: 567
best so far: 0
type: [6, 7, 1, 3, 1] 126
cases of this type: 189
best so far: 0
type: [6, 7, 3, 1, 1] 126
cases o